# Esempio di codice in Python (usando SageMath)

Questo codice analizza una versione semplificata di Poseidon 2 con crittoanalisi differenziale e lineare. Supponiamo:

- **S-box**: \( x^3 \) in \( \mathbb{F}_{2^5} \) (campo finito semplice per testing).
- **MixLayer**: Una matrice \( M \), random ma invertibile.
- **Round semplificati**: 3 (invece di 8+).

In [2]:
from sage.crypto.sbox import SBox
import numpy as np

# Parametri
n = 5  # Dimensione campo finito (2^5 per semplicità)
F = GF(2^n)
R = 3  # Round semplificati

# S-box: x^3
sbox = SBox([(F.fetch_int(x)^3).integer_representation() for x in range(2^n)])

# MixLayer: Matrice random invertibile (esempio 3x3)
M = Matrix(F, [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
assert M.is_invertible(), "M non è invertibile!"

ModuleNotFoundError: No module named 'sage'

Calcoliamo la Difference Distribution Table (DDT) per trovare differenze con alta probabilità.

In [ ]:
# Calcolo DDT
ddt = sbox.difference_distribution_table()

# Trova le differenze con probabilità massima (esclusa la diagonale)
max_prob = max(ddt[i][j] for i in range(1, 2^n) for j in range(2^n))
best_pairs = [(i, j) for i in range(1, 2^n) for j in range(2^n) if ddt[i][j] == max_prob]

print(f"Migliori coppie differenziali (Δ_in, Δ_out): {best_pairs}, Probabilità: {max_prob/2^n}")

Simuliamo un attacco per 1 round:

In [ ]:
def round_function(state, round_key):
    # Aggiungi round key
    state = [s + round_key[i] for i, s in enumerate(state)]
    # S-box
    state = [sbox(s) for s in state]
    # MixLayer
    state = M * vector(state)
    return list(state)

# Esempio di differenza Δ_in = [1, 0, 0]
delta_in = [F.fetch_int(1), F.fetch_int(0), F.fetch_int(0)]
round_key = [F.fetch_int(0)] * 3  # Chiave nulla per semplicità

# Calcola Δ_out dopo 1 round
state1 = round_function(delta_in, round_key)
state2 = round_function([F.fetch_int(0)] * 3, round_key)
delta_out = [a - b for a, b in zip(state1, state2)]

print("Differenza dopo 1 round:", [x.integer_representation() for x in delta_out])

Calcoliamo la Linear Approximation Table (LAT) per trovare maschere lineari:

In [ ]:
# Calcolo LAT
lat = sbox.linear_approximation_table()

# Trova gli sbilanciamenti massimi (escluso lo 0)
max_bias = max(abs(lat[i][j]) for i in range(1, 2^n) for j in range(1, 2^n))
best_masks = [(i, j) for i in range(1, 2^n) for j in range(1, 2^n) if abs(lat[i][j]) == max_bias]

print(f"Migliori maschere (input, output): {best_masks}, Sbilanciamento: {max_bias/2^n}")

Ecco come combinare i risultati per un attacco su 3 round:

In [ ]:
# Simulazione di un attacco differenziale su 3 round
def poseidon_3rounds(input_state, keys):
    state = input_state.copy()
    for k in range(R):
        state = round_function(state, keys[k])
    return state

# Genera coppie di input con differenza Δ_in
delta_in = [F.fetch_int(1), F.fetch_int(0), F.fetch_int(0)]
input1 = [F.fetch_int(0), F.fetch_int(0), F.fetch_int(0)]
input2 = [a + b for a, b in zip(input1, delta_in)]

keys = [[F.fetch_int(i + j) for j in range(3)] for i in range(R)]  # Chiavi random

output1 = poseidon_3rounds(input1, keys)
output2 = poseidon_3rounds(input2, keys)
delta_out = [a - b for a, b in zip(output1, output2)]

print("Differenza finale:", [x.integer_representation() for x in delta_out])